# RNN Labb (endast G)

Ni ska skapa ett nätverk som kan klassificera känslor i text. Varje text består av en mening och har en känsla kopplad till sig. Datasetet som ska användas finns på ithsdistans. Koden ska skrivas i en jupyter notebook och den ska vara välstrukturerad och hålla god kod-kvalité. (T.ex. Använd väl valda variabelnamn och kommentarer där det behövs.)

## Frågor:

- **Motivera din modellarkitektur och val av relevanta hyperparametrar.**
- **Vilka aktiveringsfunktioner har du använt? Varför?**
- **Vilken loss funktion har du använt? Varför?**
- **Har du använt någon databehandling? Motivera dina val.**
- **Har du använt någon regulariseringsteknik? Motivera.**
- **Hur har modellen utvärderats?**
- **Är prestandan bra? Varför/ varför inte?**
- **Vad hade du kunnat göra för att förbättra den ytterligare?**
- **Generativ AI har blivit stort inom text. Beskriv skillnaden mellan generativ ai och mer traditionell ai.**

*Funderingar*:

- Ska jag göra någon data processing? Hur gör man det med RNN?

- **RNN/LSTM/LSTM-GRU, vilken väljer jag?**
    - RNN lider av vanishing gradient: svårt att lära sig beroenden över långa sekvenser
    - LSTM är en avancerad variant av RNN, designad för att inte stöta på vanishing gradient
    - LSTM-GRU: variant av RNN, liknande LSTM men med en enklare struktur.
- **VAL**:
    - **Enklare/Längre Sekvenser:** Grundläggande RNN
    - **Komplex/Längre Sekvenser:** LSTM eller GRU
    - **För Balans Mellan Prestanda & Komplexitet:** GRU, ofta enklare och snabbare att träna än LSTM, men har ändå relativt lika förmåner som LSTM.

Jag tänker att jag börjar med en grundläggande RNN, då datasetet verkar inte jätte komplext.


In [38]:
import pandas as pd
from loguru import logger
import tensorflow as tf
from statistics import mean, median

df = pd.read_csv("emotions.csv")
pd.set_option('display.max_colwidth', None) # to be able to read entire text

In [ ]:
example_sentence = df["text"]

for i in range(30):
    examples = example_sentence.iloc[i]
    print(i + 1, len(examples))



In [99]:
len(example_sentence.iloc[9])

100

In [101]:
each_row = df["text"]
sentence_length = []

for row, text in enumerate(each_row.iloc):
    sentence_length.append(len(text))

print("---LENGTH OF SENTENCES IN DATASET---")
print(f"Median Length:", median(sentence_length))
print(f"Average Length:", mean(sentence_length))
print(f"Smallest Length:", min(sentence_length))
print(f"Longest Length:", max(sentence_length))

above_100_length = [length >= 100 for length in sentence_length]
is_above_100 = above_100_length.count(True)
is_not_above_100 = above_100_length.count(False)

print(f"\nRows that are ABOVE 100 characters long: {is_above_100}")
print(f"Rows that are BELOW 100 characters long: {is_not_above_100}")

print(f"\nExample sentence with 100 character length:\n", example_sentence.iloc[9])

---LENGTH OF SENTENCES IN DATASET---
Median Length: 86.0
Average Length: 96.67005
Smallest Length: 7
Longest Length: 300

Rows that are ABOVE 100 characters long: 8099
Rows that are BELOW 100 characters long: 11901

Example sentence with 100 character length:
 i the ultimate place to restore the peace to feel divine to kneel for worship and to attain hapiness


Från den här informationen så verkar datasetet relativt simpelt, alltså en grundläggande RNN bör fungera, tror jag inte behöver använda mig utan LSTM eller LSTM-GRU

In [27]:
emotions = df["label"].unique()
emotions_count = df["label"].value_counts()

for i in emotions:
    print(i)

print(f"\n{emotions_count}")

anger
sadness
joy
love
fear
surprise

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: label, dtype: int64
